<a href="https://colab.research.google.com/github/EMAT31530/ai-group-project-music-classification/blob/master/CNN%20Predict%20Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
################## Playlist Dataset Google Drive Link (also in Google Drive of ai.music.generation@gmail.com ) ########################
https://drive.google.com/drive/folders/18it5Kuzlo5OPvcxvOxrve-7-bl9t4XeE?usp=sharing

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import scipy
from scipy import misc
import glob
from PIL import Image
import os
import matplotlib.pyplot as plt
import librosa
from keras import layers
from keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, 
                          Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout)
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.initializers import glorot_uniform
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

import shutil
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img,img_to_array
import random
from google.colab import files

In [4]:
########### Load Trained Model - Doesn't seem to load properly ############
model = load_model("/content/drive/MyDrive/CNN")
#model = tf.keras.models.load_model('/content/drive/MyDrive/json_model/model2')
#model.compile(optimizer = Adam(learning_rate=0.0005),loss='categorical_crossentropy',metrics=['accuracy'])

In [5]:
data_gen=tf.keras.preprocessing.image.ImageDataGenerator()
path='/content/drive/MyDrive/playlist/images' 
image_datagentest=data_gen.flow_from_directory(path,target_size=(300,300), shuffle = False, class_mode='categorical')

Found 290 images belonging to 29 classes.


In [ ]:
path = '/content/drive/MyDrive/spectrograms3secXX/test'
genres = [f.name for f in os.scandir(path) if f.is_dir()]
genres

In [7]:
path = '/content/drive/MyDrive/playlist/images'
list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]
list_subfolder_name_with_paths = [f.name for f in os.scandir(path) if f.is_dir()]
list_subfolders_with_paths
list_subfolder_name_with_paths

['Corpse Pose',
 'All Along the Watchtower',
 'LEAVING HELL',
 'Love and or Hate Trigger',
 'Dsco',
 'Diary of a Rose',
 'Abusey Junction',
 'Circuits',
 'Like a Buoy',
 'George Street',
 'Strings of Light',
 'Suede',
 'All Caps',
 'Send Me',
 'Peanuts Grow Underground',
 'Blue in Green (feat. John Coltrane & Bill Evans)',
 'We Are Family',
 'Oblivion',
 'Slow (Loud)',
 'Track X',
 'Dark blue',
 'Bullet With a Name',
 'Wake Up',
 'Time',
 "Job's Lament",
 'Poison Tree',
 'A Message to You Rudy - 2002 Remaster',
 'Dirty Old Town',
 'Pressure Drop']

In [12]:
def predict(track_name,model):
  track_path = os.path.join(path, track_name)
  #os.remove(os.path.join(path,'.DS_Store'))
  num_files = len([name for name in os.listdir(track_path) if os.path.isfile(os.path.join(track_path, name))])
  prediction = np.zeros((num_files,10))
  i = 0
  #image = image_data.resize((288,432))
  for file_name in os.listdir(track_path):
    if not file_name == ".DS_Store":
      #print(file_name)
      file_path = os.path.join(path, track_name, file_name)
      image_data = load_img(file_path,color_mode = 'rgb' ,target_size=(128,130))
      image = img_to_array(image_data)

      image = np.reshape(image,(1,128,130,3))

      prediction[i,:] = model.predict(image, verbose=1)#/255)  #.reshape((10,)) 

      #prediction = prediction.reshape((10,)) 

    i += 1


  predict_avg = np.average(prediction, axis=0)
  predict_avg = np.around(predict_avg, decimals = 4)
  genre_class = np.argmax(predict_avg)




  
  return track_name, genre_class, predict_avg

In [ ]:
path = '/content/drive/MyDrive/playlist/images'
list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]
list_subfolders_name_with_paths = [f.name for f in os.scandir(path) if f.is_dir()]
list_subfolders_with_paths
list_subfolders_name_with_paths

num_entries = len(list_subfolders_with_paths)
  
track_name_list = []
genre_class_list = np.zeros(num_entries)
predict_avg_list = np.zeros((num_entries,10))
#predict_avg_array = np.zeros((,10))
i = 0
for subdir in os.listdir(path):
  #print(subdir)
  if not subdir == ".DS_Store":
      track_name, genre_class, predict_avg = predict(subdir,model)
      print(track_name, genre_class, predict_avg)
      track_name_list.append(track_name)
      genre_class_list[i] = (genre_class)
      predict_avg_list[i,:]=predict_avg
      print()
      i+=1
      
      

  

In [ ]:
########## Predict and Add Results to a Dataframe ################
playlist_predictions = pd.DataFrame(predict_avg_list, index = list_subfolders_name_with_paths, columns=sorted(genres))


In [30]:
genre_class_list_words = genre_class_list
for j in range(len(genres)):
  genre = sorted(genres)[j]  
  genre_class_list_words = [genre if j==i else i for i in genre_class_list_words]

  genre_class_list_words

In [32]:
playlist_predictions['Predicted Genre'] = genre_class_list_words

In [35]:
cols = list(playlist_predictions)
cols.insert(0, cols.pop(cols.index('Predicted Genre')))
playlist_predictions = playlist_predictions.loc[:, cols]

In [36]:
playlist_predictions

,Predicted Genre,blues,classical,country,disco,hiphop,jazz,metal,pop,reggae,rock
Corpse Pose,rock,0.0012,0.0674,0.0000,0.1739,0.0001,0.0000,0.0059,0.0884,0.0001,0.6629
All Along the Watchtower,rock,0.0731,0.0016,0.2933,0.0336,0.0001,0.0735,0.0005,0.0303,0.0007,0.4934
LEAVING HELL,disco,0.0000,0.0000,0.2864,0.4507,0.0000,0.0000,0.0000,0.0001,0.2555,0.0073
Love and or Hate Trigger,disco,0.0000,0.0000,0.0000,0.3853,0.3142,0.0000,0.0006,0.2884,0.0115,0.0000
Dsco,rock,0.0000,0.0060,0.0016,0.0524,0.0000,0.0001,0.0005,0.0344,0.0000,0.9050
Diary of a Rose,country,0.0001,0.0000,0.5247,0.2062,0.0000,0.0000,0.0000,0.0047,0.1696,0.0948
Abusey Junction,reggae,0.0012,0.0000,0.0324,0.0000,0.0000,0.0000,0.0000,0.0000,0.9664,0.0000
Circuits,classical,0.0000,0.9287,0.0001,0.0004,0.0000,0.0001,0.0000,0.0359,0.0002,0.0346
Like a Buoy,rock,0.0024,0.3121,0.0032,0.0123,0.0000,0.0083,0.0000,0.0072,0.0002,0.6541
George Street,rock,0.0000,0.1662,0.0000,0.0009,0.0000,0.0000,0.0006,0.0803,0.0224,0.7295


In [37]:
playlist_predictions.to_csv('predictions.csv')
files.download('predictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
################################Test Data Confidence Matrix#########################################

In [19]:
#path = '/content/drive/MyDrive/playlist/images'
path = '/content/drive/MyDrive/spectrograms3secXX/test'
list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]
list_subfolders_name_with_paths = [f.name for f in os.scandir(path) if f.is_dir()]
list_subfolders_with_paths
list_subfolders_name_with_paths

num_entries = len(list_subfolders_name_with_paths)# if os.path.isfile(os.path.join(track_path, name))])
  

In [20]:
genres = list_subfolders_name_with_paths

In [ ]:

track_name_list = []
genre_class_list = np.zeros(num_entries)
predict_avg_list = np.zeros((num_entries,10))
#predict_avg_array = np.zeros((,10))
i = 0
for subdir in os.listdir(path):
  #print(subdir)
  if not subdir == ".DS_Store":
      track_name, genre_class, predict_avg = predict(subdir,model)
      print(track_name, genre_class, predict_avg)
      track_name_list.append(track_name)
      genre_class_list[i] = (genre_class)
      predict_avg_list[i,:]=predict_avg
      print()
      i+=1
      
      

  

In [7]:
conf_matrix = pd.DataFrame(predict_avg_list, index = list_subfolders_name_with_paths, columns=sorted(list_subfolders_name_with_paths))


In [8]:
conf_matrix

,blues,classical,country,disco,hiphop,jazz,metal,pop,reggae,rock
metal,0.0437,0.0000,0.0003,0.0047,0.0116,0.0001,0.8111,0.0000,0.0000,0.1284
disco,0.0141,0.0098,0.0023,0.9527,0.0006,0.0000,0.0000,0.0002,0.0142,0.0062
pop,0.0000,0.0015,0.0104,0.0133,0.0063,0.0096,0.0000,0.9379,0.0077,0.0134
blues,0.9686,0.0000,0.0031,0.0000,0.0000,0.0068,0.0000,0.0000,0.0044,0.0071
reggae,0.0003,0.0000,0.0002,0.0055,0.0020,0.0000,0.0000,0.0000,0.9815,0.0104
classical,0.0000,0.9987,0.0001,0.0000,0.0000,0.0011,0.0000,0.0000,0.0000,0.0000
rock,0.0033,0.0003,0.0043,0.0003,0.0017,0.0011,0.0034,0.0000,0.0002,0.9854
country,0.0165,0.0002,0.9699,0.0007,0.0000,0.0008,0.0000,0.0000,0.0000,0.0119
hiphop,0.0252,0.0000,0.0065,0.0003,0.9522,0.0008,0.0000,0.0001,0.0140,0.0010
jazz,0.0020,0.0116,0.0008,0.0000,0.0000,0.9852,0.0000,0.0000,0.0000,0.0003


In [9]:
conf_matrix=conf_matrix.sort_index(ascending=True)

In [10]:
conf_matrix_row

,blues,classical,country,disco,hiphop,jazz,metal,pop,reggae,rock
blues,0.9686,0.0000,0.0031,0.0000,0.0000,0.0068,0.0000,0.0000,0.0044,0.0071
classical,0.0000,0.9987,0.0001,0.0000,0.0000,0.0011,0.0000,0.0000,0.0000,0.0000
country,0.0165,0.0002,0.9699,0.0007,0.0000,0.0008,0.0000,0.0000,0.0000,0.0119
disco,0.0141,0.0098,0.0023,0.9527,0.0006,0.0000,0.0000,0.0002,0.0142,0.0062
hiphop,0.0252,0.0000,0.0065,0.0003,0.9522,0.0008,0.0000,0.0001,0.0140,0.0010
jazz,0.0020,0.0116,0.0008,0.0000,0.0000,0.9852,0.0000,0.0000,0.0000,0.0003
metal,0.0437,0.0000,0.0003,0.0047,0.0116,0.0001,0.8111,0.0000,0.0000,0.1284
pop,0.0000,0.0015,0.0104,0.0133,0.0063,0.0096,0.0000,0.9379,0.0077,0.0134
reggae,0.0003,0.0000,0.0002,0.0055,0.0020,0.0000,0.0000,0.0000,0.9815,0.0104
rock,0.0033,0.0003,0.0043,0.0003,0.0017,0.0011,0.0034,0.0000,0.0002,0.9854


In [11]:
##############Colour Map ##################
import matplotlib.pyplot as plt

import seaborn as sns

cm = sns.light_palette("green", as_cmap=True)

s = conf_matrix_row.style.background_gradient(cmap=cm)
s


,blues,classical,country,disco,hiphop,jazz,metal,pop,reggae,rock
blues,0.968600,0.000000,0.003100,0.000000,0.000000,0.006800,0.000000,0.000000,0.004400,0.007100
classical,0.000000,0.998700,0.000100,0.000000,0.000000,0.001100,0.000000,0.000000,0.000000,0.000000
country,0.016500,0.000200,0.969900,0.000700,0.000000,0.000800,0.000000,0.000000,0.000000,0.011900
disco,0.014100,0.009800,0.002300,0.952700,0.000600,0.000000,0.000000,0.000200,0.014200,0.006200
hiphop,0.025200,0.000000,0.006500,0.000300,0.952200,0.000800,0.000000,0.000100,0.014000,0.001000
jazz,0.002000,0.011600,0.000800,0.000000,0.000000,0.985200,0.000000,0.000000,0.000000,0.000300
metal,0.043700,0.000000,0.000300,0.004700,0.011600,0.000100,0.811100,0.000000,0.000000,0.128400
pop,0.000000,0.001500,0.010400,0.013300,0.006300,0.009600,0.000000,0.937900,0.007700,0.013400
reggae,0.000300,0.000000,0.000200,0.005500,0.002000,0.000000,0.000000,0.000000,0.981500,0.010400
rock,0.003300,0.000300,0.004300,0.000300,0.001700,0.001100,0.003400,0.000000,0.000200,0.985400


In [ ]:
###################### Download colour map as Image - Not yet Completed ################################

In [ ]:
pip install dataframe_image

In [ ]:
!apt install chromium-chromedriver

In [31]:
chrome_path =  "/usr/bin/chromedriver"

In [ ]:
import numpy as np
import pandas as pd
import dataframe_image as dfi


dfi.export(s, 'conf_matrix.png')

In [17]:
plt.savefig('conf_mat.png')

<Figure size 432x288 with 0 Axes>

In [12]:
html = s.render()

In [20]:
Html_file= open("conf_mat_html","w")
Html_file.write(html)
Html_file.close()

In [ ]:
pip install wkhtmltopdf

In [ ]:
pip install imgkit

In [ ]:
import imgkit
imgkit.from_string(html, 'conf_mat.png')

In [ ]:
 
df_tracks = pd.DataFrame({'track_name':track_name_list,'genre_class':genre_class_list,})#'predict_avg':predict_avg_list})
print(df_tracks.shape)
df_tracks.head()

#arr = df_tracks['predict_avg'].to_numpy()

np.stack(arr).astype(None)

In [33]:
df_tracks = pd.DataFrame({'track_name':track_name_list,'genre_class':genre_class_list,'predict_avg':predict_avg_list})
print(df_tracks.shape)
df_tracks.head()

(10, 3)


,track_name,genre_class,predict_avg
0,metal,6,"[0.0437, 0.0, 0.0003, 0.0047, 0.0116, 0.0001, ..."
1,disco,3,"[0.0141, 0.0098, 0.0023, 0.9527, 0.0006, 0.0, ..."
2,pop,7,"[0.0, 0.0015, 0.0104, 0.0133, 0.0063, 0.0096, ..."
3,blues,0,"[0.9686, 0.0, 0.0031, 0.0, 0.0, 0.0068, 0.0, 0..."
4,reggae,8,"[0.0003, 0.0, 0.0002, 0.0055, 0.002, 0.0, 0.0,..."


In [8]:
df_tracks

,track_name,genre_class,predict_avg
0,Corpse Pose,9,"[0.0012, 0.0674, 0.0, 0.1739, 0.0001, 0.0, 0.0..."
1,All Along the Watchtower,9,"[0.0731, 0.0016, 0.2933, 0.0336, 0.0001, 0.073..."
2,LEAVING HELL,3,"[0.0, 0.0, 0.2864, 0.4507, 0.0, 0.0, 0.0, 0.00..."
3,Love and or Hate Trigger,3,"[0.0, 0.0, 0.0, 0.3853, 0.3142, 0.0, 0.0006, 0..."
4,Dsco,9,"[0.0, 0.006, 0.0016, 0.0524, 0.0, 0.0001, 0.00..."
5,Diary of a Rose,2,"[0.0001, 0.0, 0.5247, 0.2062, 0.0, 0.0, 0.0, 0..."
6,Abusey Junction,8,"[0.0012, 0.0, 0.0324, 0.0, 0.0, 0.0, 0.0, 0.0,..."
7,Circuits,1,"[0.0, 0.9287, 0.0001, 0.0004, 0.0, 0.0001, 0.0..."
8,Like a Buoy,9,"[0.0024, 0.3121, 0.0032, 0.0123, 0.0, 0.0083, ..."
9,George Street,9,"[0.0, 0.1662, 0.0, 0.0009, 0.0, 0.0, 0.0006, 0..."


In [6]:
df_tracks

,track_name,genre_class,predict_avg
0,Corpse Pose,9,"[0.0012, 0.0674, 0.0, 0.1739, 0.0001, 0.0, 0.0..."
1,All Along the Watchtower,9,"[0.0731, 0.0016, 0.2933, 0.0336, 0.0001, 0.073..."
2,LEAVING HELL,3,"[0.0, 0.0, 0.2864, 0.4507, 0.0, 0.0, 0.0, 0.00..."
3,Love and or Hate Trigger,3,"[0.0, 0.0, 0.0, 0.3853, 0.3142, 0.0, 0.0006, 0..."
4,Dsco,9,"[0.0, 0.006, 0.0016, 0.0524, 0.0, 0.0001, 0.00..."
5,Diary of a Rose,2,"[0.0001, 0.0, 0.5247, 0.2062, 0.0, 0.0, 0.0, 0..."
6,Abusey Junction,8,"[0.0012, 0.0, 0.0324, 0.0, 0.0, 0.0, 0.0, 0.0,..."
7,Circuits,1,"[0.0, 0.9287, 0.0001, 0.0004, 0.0, 0.0001, 0.0..."
8,Like a Buoy,9,"[0.0024, 0.3121, 0.0032, 0.0123, 0.0, 0.0083, ..."
9,George Street,9,"[0.0, 0.1662, 0.0, 0.0009, 0.0, 0.0, 0.0006, 0..."


In [67]:
#################Straight into dataframe ##########################
#path = '/content/drive/MyDrive/playlist/images'
path = '/content/drive/MyDrive/spectrograms3secXX/test'
list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]
list_subfolders = [f.name for f in os.scandir(path) if f.is_dir()]
list_subfolders_with_paths
list_subfolders

num_entries = len(list_subfolders_name_with_paths)# if os.path.isfile(os.path.join(track_path, name))])

df = pd.DataFrame(index=list_subfolders, columns=list_subfolders)
df = df.fillna(0) # with 0s rather than NaNs


track_name_list = []
genre_class_list = np.zeros(num_entries)
predict_avg_list = np.zeros((num_entries,10))
#predict_avg_array = np.zeros((,10))
i = 0
for subdir in os.listdir(path):
  #print(subdir)
  if not subdir == ".DS_Store":
      track_name, genre_class, predict_avg = predict(subdir,model)
      print(track_name, genre_class, predict_avg)
      df[track_name,:] = predict_avg
      #track_name_list.append(track_name)
      #genre_class_list[i] = (genre_class)
      #predict_avg_list[i,:]=predict_avg
      #print()
      #i+=1

metal 6 [4.370e-02 0.000e+00 3.000e-04 4.700e-03 1.160e-02 1.000e-04 8.111e-01
 0.000e+00 0.000e+00 1.284e-01]


TypeError: ignored